In [18]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, MultiLabelBinarizer

In [19]:
df_test = pd.read_csv("../Data/test_gr/test.csv")
df_train = pd.read_csv("../Data/train_gr/train.csv")
df_overview = pd.read_csv("../Data/train_gr/game_overview.csv")

In [20]:
#group training data by title and aggregate the mean of the user_suggestion
d = df_train.groupby([df_train["title"]], as_index=False).agg({"user_suggestion": "mean"}).sort_values(by="user_suggestion", ascending=False)

In [21]:
#join the training and overview data into an array
df_join = [d, df_overview]
#convert array into data frame
df_join = pd.concat(df_join)

#group by title and ensure that other relevant columns are preserved. Everything else is dropped
df_join = df_join.groupby(["title"]).agg({"user_suggestion": "mean", "tags": "first"}).reset_index().sort_values(by="user_suggestion", ascending=False)
df_join.head()

,title,user_suggestion,tags
20,EverQuest II,0.971014,"['Free to Play', 'Massively Multiplayer', 'RPG..."
23,Fractured Space,0.958217,"['Space', 'Multiplayer', 'Free to Play', 'PvP'..."
34,Path of Exile,0.906114,"['Free to Play', 'Action RPG', 'Hack and Slash..."
9,Creativerse,0.900407,"['Free to Play', 'Survival', 'Open World', 'Mu..."
35,PlanetSide 2,0.896186,"['Free to Play', 'Massively Multiplayer', 'FPS..."


In [22]:
#get games with nan user_suggestion values
df_nan = df_join[~(df_join["user_suggestion"] <= 1)]
df_nan_encoded = df_join[~(df_join["user_suggestion"] <= 1)]

#copy dataframe without encoded titles
df_final = df_join.copy(deep=True)
#copy dataframe with encoded titles (eventually)
df_encoded = df_join.copy(deep=True)

#drop nan values from copied dataframes
df_final = df_final.dropna()
df_encoded = df_encoded.dropna()

In [23]:
#encode titles to numeric values in df_encoded
le = LabelEncoder()
oe = OrdinalEncoder().set_output(transform="pandas")
df_encoded["title"] = le.fit_transform(df_encoded["title"])
df_nan_encoded["title"] = le.fit_transform(df_nan_encoded["title"])
df_encoded.head()

C:\Users\dylan\AppData\Local\Temp\ipykernel_10520\2187266266.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan_encoded["title"] = le.fit_transform(df_nan_encoded["title"])


,title,user_suggestion,tags
20,14,0.971014,"['Free to Play', 'Massively Multiplayer', 'RPG..."
23,16,0.958217,"['Space', 'Multiplayer', 'Free to Play', 'PvP'..."
34,21,0.906114,"['Free to Play', 'Action RPG', 'Hack and Slash..."
9,6,0.900407,"['Free to Play', 'Survival', 'Open World', 'Mu..."
35,22,0.896186,"['Free to Play', 'Massively Multiplayer', 'FPS..."


In [24]:
#make a dataframe with the tags as one big string
final_str = df_final.copy(deep=True)
encoded_str = df_encoded.copy(deep=True)

for i, row in df_final["tags"].items():
    df = row.replace("[", "").replace("]", "").replace("'", "")
    df = df.split(", ")[:5]
    
    #make tags an array of strings
    df_final["tags"][i] = df
    df_encoded["tags"][i] = df

    #make tags one big string
    final_str["tags"][i] = ' '.join(df)
    encoded_str["tags"][i] = ' '.join(df)


C:\Users\dylan\AppData\Local\Temp\ipykernel_10520\3153138693.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded["tags"][i] = df
C:\Users\dylan\AppData\Local\Temp\ipykernel_10520\3153138693.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_str["tags"][i] = ' '.join(df)
C:\Users\dylan\AppData\Local\Temp\ipykernel_10520\3153138693.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_str["tags"][i] = ' '.join(